# 기울기 형성 로직 코드

## 6개월 전체를 기준으로 타겟 형성
- 12월 데이터에만 기울기 점수가 부여
- 나머지는 기울기 점수가 0으로 나와서 이슈 발생

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# ---------------------------------------------------------
# 1. 데이터 로드 및 기초 병합
# ---------------------------------------------------------

path_approval = './카드승인정보_WideFormat_50000_2569.csv'
path_balance = './카드잔액정보_WideFormat_50000_487.csv'

months_6m = ['07', '08', '09', '10', '11', '12']
months_3m = ['10', '11', '12']

# 승인정보 로드
cols_app = ['발급회원번호'] + [f'이용금액_신용_B0M_{m}' for m in months_6m] + [f'이용건수_신용_B0M_{m}' for m in months_6m]
df_app = pd.read_csv(path_approval, usecols=lambda x: x in cols_app).fillna(0)

# 잔액정보 로드
df_bal_raw = pd.read_csv(path_balance)
cols_bal = ['발급회원번호'] + [f'잔액_B0M_{m}' for m in months_6m] + [f'잔액_현금서비스_B0M_{m}' for m in months_6m] + \
           [f'잔액_카드론_B0M_{m}' for m in months_6m] + [f'연체잔액_B0M_{m}' for m in months_6m] + [f'월중평잔_{m}' for m in months_6m]
exist_bal = [c for c in cols_bal if c in df_bal_raw.columns]
df_bal = df_bal_raw[exist_bal].fillna(0)

df_score = pd.merge(df_app, df_bal, on='발급회원번호', how='inner')

# ---------------------------------------------------------
# 2. 점수 및 타겟(Target) 생성 로직
# ---------------------------------------------------------

# 상태 점수 계산
df_score['Score_BadDebt'] = (((df_score['잔액_현금서비스_B0M_12'] - df_score['잔액_현금서비스_B0M_11']) / (df_score['잔액_현금서비스_B0M_11'] + 1) * 1.5) +
                            ((df_score['잔액_카드론_B0M_12'] - df_score['잔액_카드론_B0M_11']) / (df_score['잔액_카드론_B0M_11'] + 1) * 1.0))
df_score['Score_Delinq'] = (df_score['연체잔액_B0M_12'] * 3.0) + (df_score['연체잔액_B0M_11'] * 2.0) + (df_score['연체잔액_B0M_10'] * 1.0)
df_score['Score_Activity'] = ((df_score[[f'이용건수_신용_B0M_{m}' for m in months_3m]].sum(axis=1) * 2) - 
                               df_score[[f'이용건수_신용_B0M_{m}' for m in months_6m]].sum(axis=1)) / \
                              (df_score[[f'이용건수_신용_B0M_{m}' for m in months_6m]].sum(axis=1) + 1) * 100
df_score['Score_Asset'] = (df_score[[f'월중평잔_{m}' for m in months_3m if f'월중평잔_{m}' in df_score.columns]].mean(axis=1) / \
                           (df_score[[f'월중평잔_{m}' for m in months_6m if f'월중평잔_{m}' in df_score.columns]].mean(axis=1) + 1)) * 10
df_score['Score_Status_Total'] = (df_score['Score_BadDebt'] + df_score['Score_Delinq']) - (df_score['Score_Activity'] + df_score['Score_Asset'])

# 기울기 점수 계산
def get_slope(row, prefix):
    cols = [f"{prefix}_{m}" for m in months_6m]
    y = row[cols].values.astype(float)
    if np.sum(y) == 0: return 0
    return linregress(np.arange(len(y)), y)[0]

df_score['Slope_Spend'] = df_score.apply(lambda r: get_slope(r, '이용금액_신용_B0M'), axis=1)
df_score['Slope_Balance'] = df_score.apply(lambda r: get_slope(r, '잔액_B0M'), axis=1)
df_score['Slope_Count'] = df_score.apply(lambda r: get_slope(r, '이용건수_신용_B0M'), axis=1)

scaler = MinMaxScaler()
def norm_risk(s):
    risk = s.apply(lambda x: -x if x < 0 else 0)
    return scaler.fit_transform(risk.values.reshape(-1, 1)).flatten()

df_score['Score_Slope_Total'] = (norm_risk(df_score['Slope_Spend']) * 40 + norm_risk(df_score['Slope_Balance']) * 30 + norm_risk(df_score['Slope_Count']) * 30)
df_score['Final_Total_Score'] = (df_score['Score_Status_Total'].fillna(0) + df_score['Score_Slope_Total'].fillna(0)) * 0.5

# Risk_Count 기반 타겟 정의
df_le0 = df_score[(df_score['Slope_Spend'] <= 0) & (df_score['Slope_Balance'] <= 0) & (df_score['Slope_Count'] <= 0)].copy()
df_le0['Risk_Count'] = (df_le0[['Score_BadDebt', 'Score_Delinq']].gt(0).sum(axis=1) + 
                        df_le0['Score_Activity'].lt(0).astype(int) + 
                        df_le0['Score_Asset'].eq(0).astype(int))

churn_list = df_le0[df_le0['Risk_Count'] >= 2]['발급회원번호'].unique()
df_score['Target'] = 0
df_score.loc[df_score['발급회원번호'].isin(churn_list), 'Target'] = 1

# ---------------------------------------------------------
# 3. 원본 전체 데이터 병합 및 최종 확인
# ---------------------------------------------------------

path_raw = './50k_wideformat_all_50000_4921.csv'
df_raw = pd.read_csv(path_raw, low_memory=False)

# 병합 시 Target 컬럼 중복으로 인한 Target_x 발생을 방지하기 위해 
# 원본 데이터에 Target 컬럼이 있다면 미리 제거함
if 'Target' in df_raw.columns:
    df_raw = df_raw.drop(columns=['Target'])

# 스코어 데이터와 원본 데이터 병합
df_final = pd.merge(df_score[['발급회원번호', 'Final_Total_Score', 'Score_Status_Total', 'Score_Slope_Total', 'Target']], 
                     df_raw, 
                     on='발급회원번호', 
                     how='inner')

# 결과 확인 (이제 'Target' 컬럼을 안전하게 호출할 수 있음)
print("✅ 최종 데이터셋 구성 완료")
df_final.info()
print("-" * 30)
print(df_final['Target'].value_counts(normalize=True))

✅ 최종 데이터셋 구성 완료
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 4925 entries, 발급회원번호 to 혜택수혜율_B0M_12
dtypes: float64(1467), int64(3271), object(187)
memory usage: 1.8+ GB
------------------------------
Target
0    0.94738
1    0.05262
Name: proportion, dtype: float64


In [ ]:
# # ---------------------------------------------------------
# # 4. 최종 데이터셋 CSV 파일 저장
# # ---------------------------------------------------------

# # 한글 깨짐 방지를 위해 utf-8-sig 인코딩을 사용하며, 인덱스는 제외하고 저장함
# df_final.to_csv('Target_data_산출.csv', index=False, encoding='utf-8-sig')

In [ ]:
# df_final['Target'].value_counts(normalize=True)

Target
0    0.94738
1    0.05262
Name: proportion, dtype: float64

## 수정된 타겟
- 기울기 수정
    - 기존 6개월을 다 쓰는 기울기가 아니라, 누적 기울기 + 7월 데이터는 R12M(1년 누적데이터) 컬럼을 이용해서 기울기 점수 부여

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy.stats import linregress
import os
import tqdm  # For progress bar if available, else standard print

# =============================================================================
# [설정] 컬럼명 및 파일 경로 설정
# =============================================================================
COL_ID = '발급회원번호'
COL_DATE = '기준년월'

# 분석 변수 (Wide Format의 접미사 '_MM' 등을 뗀 순수 컬럼명 가정)
COL_SPEND = '이용금액_신용_B0M'      # 소비
COL_COUNT = '이용건수_신용_B0M'      # 빈도
COL_BALANCE = '잔액_B0M'             # 잔액
COL_CASH_ADV = '잔액_현금서비스_B0M' # 악성부채1
COL_CARD_LOAN = '잔액_카드론_B0M'    # 악성부채2
COL_DELINQ = '연체잔액_B0M'          # 리스크3
COL_AVG_BAL = '월중평잔'             # 자산

def calc_slope_long(series):
    """시계열 데이터(Series)의 선형 회귀 기울기를 계산"""
    y = series.values.astype(float)
    if len(y) < 2 or np.sum(y) == 0:
        return 0
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return 0 if np.isnan(slope) else slope

# Additional columns for R12M fallback
COL_SPEND_R12M = '이용금액_신용_R12M'
COL_COUNT_R12M = '이용건수_신용_R12M'

def calculate_churn_scores(group):
    """고객 한 명의 데이터를 받아 점수 및 Target 생성 (1개월 이상 데이터 필요)"""
    # 데이터가 아예 없는 경우
    if len(group) < 1:
        return pd.Series({
            'Score_BadDebt': 0, 'Score_Delinq': 0, 'Score_Activity': 0, 'Score_Asset': 0,
            'Score_Status_Total': 0, 'Slope_Spend': 0, 'Slope_Balance': 0, 'Slope_Count': 0
        })

    # (A) 상태 점수 (Status Score) 세부 항목 계산
    try:
        # Helper for safe indexing
        def get_val(col, idx_from_last):
            # idx_from_last: 1 for last, 2 for 2nd last...
            if len(group) >= idx_from_last:
                return group[col].iloc[-idx_from_last]
            return 0

        # 1. [부정] 악성 부채 점수 (Score_BadDebt)
        # If only 1 month, cannot compare with previous, so score is 0
        val_last = get_val(COL_CASH_ADV, 1)
        val_prev = get_val(COL_CASH_ADV, 2)
        
        loan_last = get_val(COL_CARD_LOAN, 1)
        loan_prev = get_val(COL_CARD_LOAN, 2)
        
        bad_debt_score = (
            ((val_last - val_prev) / (val_prev + 1) * 1.5) +
            ((loan_last - loan_prev) / (loan_prev + 1) * 1.0)
        )
        
        # 2. [부정] 연체 강도 점수 (Score_Delinq)
        delinq_score = (get_val(COL_DELINQ, 1) * 3.0) + (get_val(COL_DELINQ, 2) * 2.0)
        if len(group) >= 3:
            delinq_score += (get_val(COL_DELINQ, 3) * 1.0)
        
        # 3. [긍정] 활동성 점수 (Score_Activity)
        sum_r3 = group[COL_COUNT].iloc[-3:].sum()
        sum_r6 = group[COL_COUNT].sum()
        activity_score = ((sum_r3 * 2) - sum_r6) / (sum_r6 + 1) * 100
        
        # 4. [긍정] 자산 방어 점수 (Score_Asset)
        avg_r3 = group[COL_AVG_BAL].iloc[-3:].mean()
        avg_r6 = group[COL_AVG_BAL].mean()
        asset_score = (avg_r3 / (avg_r6 + 1)) * 10
        
        # >> [Total] 상태 종합 점수 (Score_Status_Total)
        score_status_total = (bad_debt_score + delinq_score) - (activity_score + asset_score)
    except:
        bad_debt_score = 0
        delinq_score = 0
        activity_score = 0
        asset_score = 0
        score_status_total = 0

    # (B) 기울기 점수 (Slope Score)
    # CASE 1: Data >= 2 months (Use linregress)
    if len(group) >= 2:
        slope_spend = calc_slope_long(group[COL_SPEND])
        slope_balance = calc_slope_long(group[COL_BALANCE])
        slope_count = calc_slope_long(group[COL_COUNT])
    
    # CASE 2: Data == 1 month (Use R12M fallback)
    else:
        # Spending Slope Proxy: Current - Monthly_Avg(R12M)
        # If current < avg, result is negative (decrease) -> Condition met
        # If current > avg, result is positive (increase) -> Condition failed
        r12m_spend = group[COL_SPEND_R12M].iloc[0] if COL_SPEND_R12M in group.columns else 0
        avg_spend = r12m_spend / 12
        slope_spend = group[COL_SPEND].iloc[0] - avg_spend
        
        # Count Slope Proxy
        r12m_count = group[COL_COUNT_R12M].iloc[0] if COL_COUNT_R12M in group.columns else 0
        avg_count = r12m_count / 12
        slope_count = group[COL_COUNT].iloc[0] - avg_count
        
        # Balance Slope: User requested to exclude this (always satisfy).
        # We set it to -1 (or any value <= 0) to ensure the condition (slope <= 0) passes.
        slope_balance = -1

    return pd.Series({
        'Score_BadDebt': bad_debt_score,
        'Score_Delinq': delinq_score,
        'Score_Activity': activity_score,
        'Score_Asset': asset_score,
        'Score_Status_Total': score_status_total,
        'Slope_Spend': slope_spend,
        'Slope_Balance': slope_balance,
        'Slope_Count': slope_count
    })

def check_churn_condition(scores):
    """Calculates Target (1 or 0) from scores series"""
    # (조건 A) 기울기 3종(소비, 잔액, 건수)이 모두 0 이하
    cond_slopes_decrease = (
        (scores['Slope_Spend'] <= 0) & 
        (scores['Slope_Balance'] <= 0) & 
        (scores['Slope_Count'] <= 0)
    )
    
    # (조건 B) 4대 위험 징후 중 1개 이상 감지 (Risk_Count >= 1)
    cond1 = scores['Score_BadDebt'] > 0
    cond2 = scores['Score_Delinq'] > 0
    cond3 = scores['Score_Activity'] < 0
    cond4 = scores['Score_Asset'] == 0
    
    risk_count = int(cond1) + int(cond2) + int(cond3) + int(cond4)
    cond_high_risk = (risk_count >= 1)
    
    return 1 if (cond_slopes_decrease and cond_high_risk) else 0

def analyze_rolling_churn(file_path):
    print(f"\n[Info] 파일 로드 및 분석 시작: {file_path}")
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return

    # 1. Load Data
    try:
        df = pd.read_csv(file_path, low_memory=False)
        # Fill missing cols with 0 if needed
        # Added R12M columns to required list
        required_cols = [
            COL_ID, COL_DATE, COL_SPEND, COL_COUNT, COL_BALANCE, 
            COL_CASH_ADV, COL_CARD_LOAN, COL_DELINQ, COL_AVG_BAL,
            COL_SPEND_R12M, COL_COUNT_R12M
        ]
        for c in required_cols:
            if c not in df.columns: df[c] = 0
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        return

    # Sort
    df.sort_values(by=[COL_ID, COL_DATE], inplace=True)
    
    # Group by ID
    grouped = df.groupby(COL_ID)
    
    results = []
    
    print(" - 고객별 Rolling Analysis 진행 중... (데이터 최소 1개월 기준, 1개월 시 R12M 보완)")
    
    count_churners = 0
    total_processed = 0
    
    for cust_id, group in grouped:
        total_processed += 1
        # Minimum 1 months required logic now supported
        if len(group) < 1:
            continue
            
        # 1. "현재 시점"의 이탈 여부를 확인
        # (주의: 사용자가 원하는 것은 '현재 이탈자인 사람'의 과거 지속기간 확인이므로,
        # 가장 마지막 달이 Churn이어야 분석 대상이 됨)
        current_scores = calculate_churn_scores(group)
        is_current_churn = check_churn_condition(current_scores)
        
        if is_current_churn == 1:
            count_churners += 1
            
            # 2. 이탈자라면, 과거로 역추적 (Rolling)
            consecutive_months = 1 # Start with 1 (the current month)
            
            # Max lookback
            # If len=6, loops i=1 to 5 (check len=5 down to len=1)
            # If len=1, loop range(1, 1) -> Empty loop (correct, duration=1)
            max_lookback = len(group) - 1
            
            for i in range(1, max_lookback + 1):
                past_group = group.iloc[:-i] # Remove last i rows
                past_scores = calculate_churn_scores(past_group)
                is_past_churn = check_churn_condition(past_scores)
                
                if is_past_churn == 1:
                    consecutive_months += 1
                else:
                    break # Break chain
            
            results.append({
                'Cust_ID': cust_id,
                'Churn_Duration_Months': consecutive_months
            })
            
        # if total_processed % 1000 == 0:
        #     print(f"   ... {total_processed}명 처리 완료 (발견된 이탈자: {count_churners}명)")

    # Output Results
    if len(results) == 0:
        print("❌ 분석된 이탈자가 없습니다.")
        return

    df_res = pd.DataFrame(results)
    
    print("\n" + "="*50)
    print(f"📊 이탈자 Rolling 분석 결과 (대상 파일: {os.path.basename(file_path)})")
    print("="*50)
    print(f" - 총 분석 고객 수: {total_processed}명")
    print(f" - 최종 시점 이탈자 수: {len(df_res)}명 ({len(df_res)/total_processed*100:.2f}%)")
    print("-" * 30)
    print(" [이탈 징후 지속 기간 통계]")
    print(df_res['Churn_Duration_Months'].describe())
    print("-" * 30)
    print(" [기간별 분포 (상위 10개)]")
    print(df_res['Churn_Duration_Months'].value_counts().sort_index(ascending=False).head(10))
    
    # Save detailed results
    save_path = f"churn_duration_results_{os.path.basename(file_path)}"
    # df_res.to_csv(save_path, index=False)
    print(f"\n✅ 상세 결과 저장 완료: {save_path}")


[Info] 파일 로드 및 분석 시작: ../../../260108/general_combined_part1.csv
 - 고객별 Rolling Analysis 진행 중... (데이터 최소 1개월 기준, 1개월 시 R12M 보완)


C:\Users\johnh\AppData\Local\Temp\ipykernel_14832\2085201135.py:75: RuntimeWarning: divide by zero encountered in scalar divide
  activity_score = ((sum_r3 * 2) - sum_r6) / (sum_r6 + 1) * 100



📊 이탈자 Rolling 분석 결과 (대상 파일: general_combined_part1.csv)
 - 총 분석 고객 수: 84000명
 - 최종 시점 이탈자 수: 33455명 (39.83%)
------------------------------
 [이탈 징후 지속 기간 통계]
count    33455.000000
mean         2.197459
std          1.617552
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: Churn_Duration_Months, dtype: float64
------------------------------
 [기간별 분포 (상위 10개)]
Churn_Duration_Months
6     2872
5     1934
4     1438
3     3083
2     7485
1    16643
Name: count, dtype: int64

✅ 상세 결과 저장 완료: churn_duration_results_general_combined_part1.csv


In [ ]:
TEST_FILE = '../../../260108/general_combined_part1.csv'
analyze_rolling_churn(TEST_FILE)